In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob
import seaborn as sns

## Read from `~/metrics/epoch_metrics/` directory

In [2]:
files = glob.glob("../../metrics/epoch_metrics/*parquet")

dataframes = []

for file in files:
    df = pd.read_parquet(file)
    
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

In [3]:
df["per_layer"] = df['total_rounds'].apply(lambda x: isinstance(x, np.ndarray))

In [4]:
#df = pd.read_parquet("../../metrics/epoch_metrics/")

# Hyper-Parameters

`bias` `NaN` -> no bias

`bias` $ \in (0, 1)$ -> `bias`% of each client's dataset is biased (from sorted sequence of MNIST) - rest is good

`bias` $= -1$ -> MNIST label 0 is sorted, placed at the start, and clients get assigned sequentially from this dataset. Only one label is biased to however many clients can get it assigned to them.

`bias` $= -2$ -> MNIST label 8 is sorted, placed at the start, and clients get assigned sequentially from this dataset. Only one label is biased to however many clients can get it assigned to them.

In [5]:
one_label_bias = {
    -1: 0,
    -2: 8
}

In [6]:
aggr = {
    'avg': '',
    'wavg_drifts': ' wavg'
}

In [7]:
per_layer_dict = {
    False: '',
    True: ' per-layer'
}

## All tests summary

Note that this is a summary. 

Notes:
1. `synchronous` method implies `theta` equal to `0.0` and the reverse.
2. The stopping criterion (epochs at the moment) is not the same for all combination, in fact, it changes.

In [13]:
df_lenet = df[df['nn_name'] == 'LeNet-5']

In [14]:
for col in ['dataset_name', 'fda_name', 'nn_name', 'num_clients', 'batch_size', 'num_steps_until_rtc_check', 'theta', 'bias', 'aggr_scheme']:
    print(f"{col}: {sorted(list(df_lenet[col].unique()))}")

dataset_name: ['MNIST']
fda_name: ['gm', 'linear', 'naive', 'sketch', 'synchronous']
nn_name: ['LeNet-5']
num_clients: [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
batch_size: [32, 64, 128, 256]
num_steps_until_rtc_check: [1]
theta: [0.0, 0.5, 1.0, 1.5, 2.0, 3.0, 5.0, 7.0, 10.0, 12.0, 15.0]
bias: [nan, -2.0, -1.0, 0.3, 0.6, 0.9]
aggr_scheme: ['avg']


## Query All tests

In [15]:
test_combinations = df.groupby(['dataset_name', 'nn_name', 'fda_name', 'num_steps_until_rtc_check', 'batch_size', 'theta', 'num_clients', 'aggr_scheme', 'bias', 'per_layer'], dropna=False)['epoch'].max().reset_index()

In [16]:
test_combinations[
    (test_combinations['nn_name'] == 'AdvancedCNN') &
    (test_combinations['fda_name'] == 'naive') &
    (test_combinations['theta'] == 20.0) &
    (test_combinations['num_clients'] == 5)
]

,dataset_name,nn_name,fda_name,num_steps_until_rtc_check,batch_size,theta,num_clients,aggr_scheme,bias,per_layer,epoch
816,MNIST,AdvancedCNN,naive,1,32,20.0,5,avg,-2.0,False,100
817,MNIST,AdvancedCNN,naive,1,32,20.0,5,avg,-1.0,False,100
818,MNIST,AdvancedCNN,naive,1,32,20.0,5,avg,0.9,False,100
819,MNIST,AdvancedCNN,naive,1,32,20.0,5,avg,NaN,False,100


# Helpful new Dataframe metrics

### Add Helpful model metrics

In [17]:
df['model_bytes'] = df['nn_num_weights'] * 4

### Add Helpful FDA method metrics

In [18]:
def fda_local_state_bytes(row):
    if row['fda_name'] == "naive":
        return 4
    if row['fda_name'] == "linear":
        return 8
    if row['fda_name'] == "sketch":
        return row['sketch_width'] * row['sketch_depth'] * 4 + 4
    if row['fda_name'] == "synchronous":
        return 0
    if row['fda_name'] == 'gm':
        return 0.125  # one bit

In [19]:
df['local_state_bytes'] = df.apply(fda_local_state_bytes, axis=1)

### Add Total Steps

total steps (a single fda step might have many normal SGD steps, batch steps)

In [20]:
df['total_steps'] = df['total_fda_steps'] * df['num_steps_until_rtc_check']

### Add communication metrics

In [21]:
from fdavg.models.advanced_cnn import get_compiled_and_built_advanced_cnn

adv = get_compiled_and_built_advanced_cnn((None, 28, 28), (28, 28, 1), 10)
adv_layer_count = [len(v) for v in adv.per_layer_trainable_vars_as_vector()]

2023-11-28 09:49:38.255376: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 09:49:38.483792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 09:49:38.483896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 09:49:38.523881: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 09:49:38.610141: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 09:49:38.611742: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [22]:
adv_layer_count

[640, 36928, 73856, 147584, 295168, 590080, 1180160, 262656, 5130]

The communication bytes exchanged for model synchronization. Remember that the Clients send their models to the Server and the Server sends the global model back. This happens at the end of every round.

In [23]:
len(df)

771000

In [24]:
def model_bytes_exchanged(row):
    if row['per_layer']:
        return sum((layer_num_weights * 4) * layer_rounds * row['num_clients'] * 2 for layer_num_weights, layer_rounds in zip(adv_layer_count, row['total_rounds']))
    
    return row['total_rounds'] * row['model_bytes'] * row['num_clients'] * 2

In [25]:
df['model_bytes_exchanged'] = df.apply(model_bytes_exchanged, axis=1)

The communication bytes exchanged for monitoring the variance. This happens at the end of every FDA step which consists of `num_steps_until_rtc_check` number of steps. 

In [26]:
def monitoring_bytes_exchanged(row):
    if row['per_layer']:
        return row['local_state_bytes'] * row['total_fda_steps'] * row['num_clients'] * len(adv_layer_count)
    
    return row['local_state_bytes'] * row['total_fda_steps'] * row['num_clients']

In [27]:
df['monitoring_bytes_exchanged'] = df.apply(monitoring_bytes_exchanged, axis=1)

The total communication bytes exchanged in the whole Federated Learning lifecycle.

In [28]:
df['total_communication_bytes'] = df['model_bytes_exchanged'] + df['monitoring_bytes_exchanged']

In [29]:
df['total_communication_gb'] = df['total_communication_bytes'] / 10**9

# HyperParameter ranking

### LeNet-5
On 2 `Nvidia A10`:
1. Batch Size = 32 -> 6.613 ms ± 0.128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 7.509 ms ± 0.065 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 8.02 ms ± 0.099 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 9.258 ms ± 0.336 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## AdvancedCNN

On 2 `Nvidia A10`:
1. *Batch Size* = 32 -> 8.853 ms ± 0.0917 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2. *Batch Size* = 64 -> 10.325 ms ± 0.215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3. *Batch Size* = 128 -> 11.989 ms ± 0.134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4. *Batch Size* = 256 -> 16.47 ms ± 0.294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [30]:
step_ms = {
    ("AdvancedCNN", 32): 8.853,
    ("AdvancedCNN", 64): 10.325,
    ("AdvancedCNN", 128): 11.989,
    ("AdvancedCNN", 256): 16.47,
    ("LeNet-5", 32): 6.613,
    ("LeNet-5", 64): 7.509,
    ("LeNet-5", 128): 8.02,
    ("LeNet-5", 256): 9.258
}

In [31]:
import numpy as np

def cpu_time_cost(row):
    """ Total cpu time cost in (sec).
    A single `step` means each client performed a single `step` 
    """
    return row['total_steps'] * step_ms[(row['nn_name'], row['batch_size'])] / 1000

def communication_time_cost(num_clients, total_communication_bytes, comm_model):
    """ Assuming channel is 1Gbps """

    total_communication_gbit = total_communication_bytes * 8e-9

    if comm_model == 'common_channel':
        
        return ((num_clients - 1) / num_clients) * total_communication_gbit    # sec

    if comm_model == 'hypercube':

        return (np.ceil(np.log(num_clients)) / num_clients) * total_communication_gbit   # sec

In [32]:
df['cpu_time_cost'] = df.apply(cpu_time_cost, axis=1)

In [33]:
df['hypercube_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'hypercube')

In [34]:
df['common_channel_communication_time_cost'] = communication_time_cost(df['num_clients'], df['total_communication_bytes'], 'common_channel')

In [35]:
df['hypercube_time_cost'] = df['cpu_time_cost'] + df['hypercube_communication_time_cost']

In [36]:
df['common_channel_time_cost'] = df['cpu_time_cost'] + df['common_channel_communication_time_cost']

In [37]:
df['hypercube_comm_cpu_time_ratio'] = df['hypercube_communication_time_cost'] / df['cpu_time_cost']

In [38]:
df['common_channel_comm_cpu_time_ratio'] = df['common_channel_communication_time_cost'] / df['cpu_time_cost']

# Plots about cost

In [39]:
# Define styles for each fda_name
fda_styles = {
    'naive': 'o-r',
    'linear': 's-g',
    'sketch': '^-b',
    'synchronous': 'x-c'
}
fda_names = ['gm', 'naive', 'linear', 'sketch', 'synchronous']

In [40]:
import matplotlib

num_clients_values = sorted(df['num_clients'].unique())
cmap = matplotlib.colormaps['tab20b']
colors_dict = {
    num_clients: color 
    for num_clients, color in zip(num_clients_values, cmap(np.linspace(0, 1, len(num_clients_values))))
}

## KDE Helper

In [41]:
"""
sns_params = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': True,
    'alpha': 0.2
}
"""
sns_params = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': False,
    'alpha': 1
}

sns_params_biases = {
    'bw_method': 'scott',
    'bw_adjust': 0.7,
    'fill': False,
    'alpha': 0.8
}

base_colors = {
    'gm': 'blue',
    'naive': 'orange',
    'linear': 'green',
    'sketch': 'red',
    'synchronous': 'purple',
    'sketch wavg': 'black'
}

base_colors_per_layer = {
    'gm': 'blue',
    'naive': 'orange',
    'linear': 'green',
    'sketch': 'red',
    'synchronous': 'purple',
    'naive per-layer': 'black'
}

sync_colors = {
    32: 'rebeccapurple',
    64: 'darkorchid',
    128: 'mediumorchid',
    256: 'plum'
}

In [42]:
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 14

## Total time cost with accuracy

### KDE

In [43]:
def kde_time_cost(df, filename, x_log=True):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    plt.rcParams['font.size'] = 20
    plt.rcParams['legend.fontsize'] = 12
    
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))
        
        for fda_name in fda_names:

            common_channel_df = df_bias[df_bias['fda_name'] == fda_name]['common_channel_time_cost']
            sns.kdeplot(common_channel_df, label=fda_name, ax=axs[0], color=base_colors[fda_name], log_scale=log_scale, **sns_params)

            hypercube_df = df_bias[df_bias['fda_name'] == fda_name]['hypercube_time_cost']
            sns.kdeplot(hypercube_df, label=fda_name, ax=axs[1], color=base_colors[fda_name], log_scale=log_scale, **sns_params)


        if not x_log:
            axs[0].set_xlim(left=0)
        axs[0].set_xlabel('Time Cost (sec)')
        axs[0].set_ylabel('Density')
        axs[0].legend()
        axs[0].set_title("Common Channel Communication Model")
        axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[0].legend()

        if not x_log:
            axs[1].set_xlim(left=0)

        axs[1].set_xlabel('Time Cost (sec)')
        axs[1].set_ylabel('Density')
        axs[1].legend()
        axs[1].set_title("Hypercube Communication Model")
        axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[1].legend()
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        fig.suptitle(info+bias_title)
        
        plt.tight_layout()

        pdf.savefig(fig)

        plt.close(fig)
    
        
    pdf.close()
    
    plt.rcParams['font.size'] = 14
    plt.rcParams['legend.fontsize'] = 12

## Total Communication cost (in gb) with accuracy (scatter)

### KDE

In [44]:
def kde_communication_cost(df, filename, x_log=True):

    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        plt.figure(figsize=(10, 6))

        avg_communications_dict = {}
        avg_info = []

        for fda_name in fda_names:
            fda_data_df = df_bias[df_bias['fda_name'] == fda_name]['total_communication_gb']
            
            if fda_data_df.empty:
                continue
            
            avg_communications_dict[fda_name] = fda_data_df.mean()
            avg_info.append(f'{fda_name}: {avg_communications_dict[fda_name]:.2f} GB')

            # Plotting only the KDE using kdeplot
            sns.kdeplot(fda_data_df, label=fda_name, log_scale=log_scale, color=base_colors[fda_name], **sns_params)


        text = "Average Communication:\n" + '\n'.join(avg_info)
        # Add the text annotation inside the plot
        plt.text(0.02, 0.97, text, transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.4))


        if not x_log:
            plt.xlim(left=0)

        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        plt.xlabel('Communication (GB)')
        plt.ylabel('Density')
        #plt.legend()  # We'll be moving this
        plt.gca().legend(loc='best', bbox_to_anchor=(0, 0.1, 1, 0.67))
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        plt.suptitle(info+bias_title)

        plt.tight_layout()

        pdf.savefig(plt.gcf()) # Save the current figure

        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
    pdf.close()

## Total CPU time (in Seconds) with accuracy

### KDE

In [45]:
def kde_cpu_time_cost(df, filename, x_log=False):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        plt.figure(figsize=(10, 6))

        for fda_name in fda_names:
            fda_data_df = df_bias[df_bias['fda_name'] == fda_name]['cpu_time_cost']
            
            if fda_data_df.empty:
                continue

            # Plotting only the KDE using kdeplot
            sns.kdeplot(fda_data_df, label=fda_name, log_scale=log_scale, color=base_colors[fda_name], **sns_params)

        if not x_log:
            plt.xlim(left=0)
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        plt.xlabel('CPU time cost (sec)')
        plt.ylabel('Density')
        plt.legend()
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        plt.suptitle(info+bias_title)
        
        plt.tight_layout()

        pdf.savefig(plt.gcf()) # Save the current figure
        
        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
    pdf.close()

## Total CPU cost / Comm Cost (time) - ratio 

In [46]:
def kde_comm_cpu_time_cost_ratio(df, filename, x_log=True):
    
    if x_log:
        log_scale = (True, False)
    else:
        log_scale = False
        
    plt.rcParams['font.size'] = 20
    plt.rcParams['legend.fontsize'] = 12
    
    pdf = PdfPages(filename)
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        if len(df_bias['fda_name'].unique()) < 2:
            continue
    
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))

        for fda_name in fda_names:
            df_fda = df_bias[df_bias['fda_name'] == fda_name]
            
            if df_fda.empty:
                continue
            
            common_channel_ratio_df = df_fda['cpu_time_cost'] / df_fda['common_channel_communication_time_cost']
            sns.kdeplot(common_channel_ratio_df, label=fda_name, ax=axs[0], color=base_colors[fda_name], log_scale=log_scale, **sns_params)

            hypercube_ratio_df = df_fda['cpu_time_cost'] / df_fda['hypercube_communication_time_cost']
            sns.kdeplot(hypercube_ratio_df, label=fda_name, ax=axs[1], color=base_colors[fda_name], log_scale=log_scale, **sns_params)


        if not x_log:
            axs[0].set_xlim(left=0)
        axs[0].set_xlabel('CPU Time Cost / Comm. Time Cost')
        axs[0].set_ylabel('Density')
        axs[0].legend()
        axs[0].set_title("Common Channel Communication Model")
        axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[0].legend()

        if not x_log:
            axs[1].set_xlim(left=0)

        axs[1].set_xlabel('CPU Time Cost / Comm. Time Cost')
        axs[1].set_ylabel('Density')
        axs[1].legend()
        axs[1].set_title("Hypercube Communication Model")
        axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
        axs[1].legend()
        
        fda_df = df_bias[df_bias['fda_name'] != 'synchronous']
        sync_df = df_bias[df_bias['fda_name'] == 'synchronous']
        info = f"gm, naive, linear, sketch - bs : {fda_df['batch_size'].unique()}  ,  synchronous - bs: {sync_df['batch_size'].unique()}\n"
        fig.suptitle(info+bias_title)
        
        plt.tight_layout()

        pdf.savefig(fig)

        plt.close(fig)
    
        
    pdf.close()
    
    plt.rcParams['font.size'] = 14
    plt.rcParams['legend.fontsize'] = 12

## Keep Hyper-parameters fixed and plot for filtered

In [47]:
def fda_methods_clients_time_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 

                if filtered_df.empty:
                    continue

                fig, axs = plt.subplots(1, 2, figsize=(20, 6))

                for fda_name in fda_names:

                    if fda_name == 'synchronous':
                        continue

                    fda_data = filtered_df[filtered_df['fda_name'] == fda_name]

                    if fda_data.empty:
                        continue

                    axs[0].plot(fda_data['num_clients'], fda_data['common_channel_time_cost'], marker='o', color=base_colors[fda_name], label=fda_name, markersize=3)
                    axs[1].plot(fda_data['num_clients'], fda_data['hypercube_time_cost'], marker='o', color=base_colors[fda_name], label=fda_name, markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous')] 

                if not synchronous_data.empty:
                    axs[0].plot(synchronous_data['num_clients'], synchronous_data['common_channel_time_cost'], marker='o', label='synchronous', color=base_colors['synchronous'], markersize=3)
                    axs[1].plot(synchronous_data['num_clients'], synchronous_data['hypercube_time_cost'], marker='o', label='synchronous', color=base_colors['synchronous'], markersize=3)

                if not axs[0].has_data():
                    plt.close(fig)
                    continue

                # Set xticks every 5 units based on available values
                min_clients = 5
                max_clients = 60

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                axs[0].set_yscale('log')
                axs[0].set_xticks(x_ticks)
                axs[0].set_xlabel('Number of clients')
                axs[0].legend()
                axs[0].set_title("Common Channel Communication Model")
                axs[0].set_ylabel('Time Cost')
                axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                axs[1].set_yscale('log')
                axs[1].set_xticks(x_ticks)
                axs[1].set_xlabel('Number of clients')
                axs[1].legend()
                axs[1].set_title("Hypercube Communication Model")
                axs[1].set_ylabel('Time Cost')
                axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                fig.suptitle(title)

                plt.tight_layout()

                pdf.savefig(fig)

                plt.close(fig)
    pdf.close()

In [48]:
def fda_methods_clients_comm_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]
    
    aggr_schemes = df['aggr_scheme'].unique()  # new

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 

                if filtered_df.empty:
                    continue

                plt.figure(figsize=(10, 6))
                
                for aggr_scheme in aggr_schemes:
                    
                    aggr_df = filtered_df[filtered_df['aggr_scheme'] == aggr_scheme]
                    
                    for fda_name in fda_names:
                        
                        name = f"{fda_name}{aggr[aggr_scheme]}"

                        if fda_name == 'synchronous':
                            continue

                        fda_data = aggr_df[aggr_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        plt.plot(fda_data['num_clients'], fda_data['total_communication_gb'], marker='o', color=base_colors[name], label=name, markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous')] 

                if not synchronous_data.empty:
                    plt.plot(synchronous_data['num_clients'], synchronous_data['total_communication_gb'], marker='o', label='synchronous', color=base_colors['synchronous'], markersize=3)

                # Set xticks every 5 units based on available values
                min_clients = 5
                max_clients = 60

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                plt.yscale('log')
                plt.xticks(x_ticks)
                plt.xlabel('Number of clients')
                plt.legend()
                plt.ylabel('Communication (GB)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf()) # Save the current figure
        
                # Close the current figure to prevent it from being displayed in the notebook
                plt.close()
    pdf.close()

In [49]:
def fda_methods_clients_cpu_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    theta_values = sorted(df['theta'].unique())[1:]
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 
                    
                if filtered_df.empty:
                    continue

                plt.figure(figsize=(10, 6))

                for fda_name in fda_names:

                    if fda_name == 'synchronous':
                        continue

                    fda_data = filtered_df[filtered_df['fda_name'] == fda_name]

                    if fda_data.empty:
                        continue

                    plt.plot(fda_data['num_clients'], fda_data['cpu_time_cost'], marker='o', color=base_colors[fda_name], label=fda_name, markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous')] 

                if not synchronous_data.empty:
                    plt.plot(synchronous_data['num_clients'], synchronous_data['cpu_time_cost'], marker='o', label='synchronous', color=base_colors['synchronous'], markersize=3)

                # Set xticks every 5 units based on available values
                min_clients = 5
                max_clients = 60

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                #plt.yscale('log')
                plt.xticks(x_ticks)
                plt.xlabel('Number of clients')
                plt.legend()
                plt.ylabel('CPU Time Cost (sec)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf()) # Save the current figure
        
                # Close the current figure to prevent it from being displayed in the notebook
                plt.close()
    pdf.close()

In [50]:
def fda_methods_theta_time_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    num_clients_values = sorted(df['num_clients'].unique())
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]

        for batch_size in batch_size_values:
            for num_clients in num_clients_values:

                filtered_df = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['num_clients'] == num_clients)] 

                if filtered_df[filtered_df['fda_name'] != 'synchronous'].empty:
                    continue

                fig, axs = plt.subplots(1, 2, figsize=(20, 6))

                for fda_name in fda_names:

                    if fda_name == 'synchronous':
                        continue

                    fda_data = filtered_df[filtered_df['fda_name'] == fda_name]

                    if fda_data.empty:
                        continue

                    axs[0].plot(fda_data['theta'], fda_data['common_channel_time_cost'], marker='o', label=fda_name, color=base_colors[fda_name], markersize=3)
                    axs[1].plot(fda_data['theta'], fda_data['hypercube_time_cost'], marker='o', label=fda_name, color=base_colors[fda_name], markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous') & (df_bias['num_clients'] == num_clients)] 

                if not synchronous_data.empty:
                    axs[0].axhline(y=synchronous_data['common_channel_time_cost'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)
                    axs[1].axhline(y=synchronous_data['hypercube_time_cost'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)

                if not axs[0].has_data():
                    plt.close(fig)
                    continue

                axs[0].set_yscale('log')
                axs[0].set_xlabel('Theta')
                axs[0].legend()
                axs[0].set_title("Common Channel Communication Model")
                axs[0].set_ylabel('Time Cost')
                axs[0].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                axs[1].set_yscale('log')
                axs[1].set_xlabel('Theta')
                axs[1].legend()
                axs[1].set_title("Hypercube Communication Model")
                axs[1].set_ylabel('Time Cost')
                axs[1].grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , Num Clients : {num_clients} , {bias_title}'
                fig.suptitle(title)

                plt.tight_layout()

                pdf.savefig(fig)

                plt.close(fig)
    pdf.close()

In [51]:
def fda_methods_theta_comm_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    num_clients_values = sorted(df['num_clients'].unique())
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]

        for batch_size in batch_size_values:
            for num_clients in num_clients_values:

                filtered_df = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['num_clients'] == num_clients)] 

                if filtered_df[filtered_df['fda_name'] != 'synchronous'].empty:
                    continue

                plt.figure(figsize=(10, 6))

                for fda_name in fda_names:

                    if fda_name == 'synchronous':
                        continue

                    fda_data = filtered_df[filtered_df['fda_name'] == fda_name]

                    if fda_data.empty:
                        continue

                    plt.plot(fda_data['theta'], fda_data['total_communication_gb'], marker='o', label=fda_name, color=base_colors[fda_name], markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous') & (df_bias['num_clients'] == num_clients)] 

                if not synchronous_data.empty:
                    plt.axhline(y=synchronous_data['total_communication_gb'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)

                plt.yscale('log')
                plt.xlabel('Theta')
                plt.legend()
                plt.ylabel('Communication Cost (GB)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , Num Clients : {num_clients} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf())

                plt.close()
    pdf.close()

In [52]:
def fda_methods_theta_cpu_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = sorted(df['batch_size'].unique())
    num_clients_values = sorted(df['num_clients'].unique())
    
    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]

        for batch_size in batch_size_values:
            for num_clients in num_clients_values:

                filtered_df = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['num_clients'] == num_clients)] 

                if filtered_df[filtered_df['fda_name'] != 'synchronous'].empty:
                    continue

                plt.figure(figsize=(10, 6))

                for fda_name in fda_names:

                    if fda_name == 'synchronous':
                        continue

                    fda_data = filtered_df[filtered_df['fda_name'] == fda_name]

                    if fda_data.empty:
                        continue

                    plt.plot(fda_data['theta'], fda_data['cpu_time_cost'], marker='o', label=fda_name, color=base_colors[fda_name], markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous') & (df_bias['num_clients'] == num_clients)] 

                if not synchronous_data.empty:
                    plt.axhline(y=synchronous_data['cpu_time_cost'].iloc[0], label='synchronous', marker='o', color=base_colors['synchronous'], markersize=3)

                #plt.yscale('log')
                plt.xlabel('Theta')
                plt.legend()
                plt.ylabel('CPU Time Cost (sec)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , Num Clients : {num_clients} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf())

                plt.close()
    pdf.close()

# Per accuracy

In [53]:
def accuracies_plots_clients(df, filename, accuracies):
    pdf = PdfPages(filename)
    
    theta_values = sorted(df['theta'].unique())[1:]

    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    clients = list(range(5, 60 + 1, 5))
    
    batch_sizes = df['batch_size'].unique()

    cmap = matplotlib.colormaps['tab20b']
    colors_dict = {
        num_clients: color 
        for num_clients, color in zip(clients, cmap(np.linspace(0, 1, len(num_clients_values))))
    }

    for bias in biases:

        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'

        df_bias = df[mask]
        
        # 1. Synchronous ALL
        
        df_sync = df_bias[df_bias['fda_name'] == 'synchronous']
        
        for bs in sorted(df_sync['batch_size'].unique()):
            
            bs_df = df_sync[df_sync['batch_size'] == bs]

            dataframes_for_acc = []

            plt.figure(figsize=(10, 6))

            for acc in accuracies:

                filtered_df = bs_df[(bs_df.accuracy > acc)]
                idx = filtered_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta', 'bias', 'aggr_scheme'], dropna=False)['epoch'].idxmin()
                filtered_df = filtered_df.loc[idx]

                dataframes_for_acc.append(filtered_df)

            df_concat = pd.concat(dataframes_for_acc)

            for num_clients in clients:
                client_df = df_concat[df_concat['num_clients'] == num_clients]

                plt.plot(client_df['total_communication_gb'], client_df['accuracy'], marker='o', color=colors_dict[num_clients], label=num_clients, markersize=3)

            if df_concat.empty:
                plt.close()
                continue

            plt.xscale('log')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.xlabel('Communication Cost (GB)')
            plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

            title = f'Batch Size : {bs} , {bias_title}'
            plt.title(title)
            plt.suptitle('synchronous')

            pdf.savefig(plt.gcf())

            plt.close()

        #plt.figure(figsize=(10, 6))
        
        not_sync_df = df_bias[df_bias['fda_name'] != 'synchronous']

        for theta in theta_values:
            theta_df = not_sync_df[not_sync_df['theta'] == theta]
            
            for bs in sorted(theta_df['batch_size'].unique()):
                bs_df = theta_df[theta_df['batch_size'] == bs]

                for fda_name in fda_names:

                    fda_df = bs_df[bs_df['fda_name'] == fda_name]

                    dataframes_for_acc = []

                    plt.figure(figsize=(10, 6))

                    for acc in accuracies:

                        filtered_df = fda_df[(fda_df.accuracy > acc)]
                        idx = filtered_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta', 'bias'], dropna=False)['epoch'].idxmin()
                        filtered_df = filtered_df.loc[idx]

                        dataframes_for_acc.append(filtered_df)

                    df_concat = pd.concat(dataframes_for_acc)

                    for num_clients in clients:
                        client_df = df_concat[df_concat['num_clients'] == num_clients]

                        plt.plot(client_df['total_communication_gb'], client_df['accuracy'], marker='o', color=colors_dict[num_clients], label=num_clients, markersize=3)

                    if df_concat.empty:
                        plt.close()
                        continue

                    plt.xscale('log')
                    plt.ylabel('Accuracy')
                    plt.legend()
                    plt.xlabel('Communication Cost (GB)')
                    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                    title = f'$Theta$ : {theta} , Batch Size: {bs} , {bias_title}'
                    plt.title(title)
                    plt.suptitle(fda_name)

                    pdf.savefig(plt.gcf())

                    plt.close()
    pdf.close()

In [54]:
def accuracies_plots(df, filename, accuracies):
    pdf = PdfPages(filename)

    biases = sorted(df['bias'].unique(), reverse=True)[::-1]

    for bias in biases:

        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'

        df_bias = df[mask]
        
        for bs in sorted(df_bias['batch_size'].unique()):

            bs_df = df_bias[df_bias['batch_size'] == bs]
                
            for theta in sorted(bs_df['theta'].unique())[1:]:

                theta_df = bs_df[bs_df['theta'] == theta]

                for client in sorted(theta_df['num_clients'].unique()):

                    client_df = theta_df[theta_df['num_clients'] == client]

                    if client_df.empty:
                        continue

                    plt.figure(figsize=(10, 6))

                    for fda_name in fda_names:

                        if fda_name == 'synchronous':
                            continue

                        fda_df = client_df[client_df['fda_name'] == fda_name]

                        if fda_df.empty:
                            continue

                        dataframes_for_acc = []
                        have = False

                        for acc in accuracies:

                            filtered_df = fda_df[(fda_df.accuracy > acc)]

                            if filtered_df.empty:
                                continue
                                
                            have = True

                            idx = filtered_df['epoch'].idxmin()
                            filtered_df = filtered_df.loc[idx]

                            dataframes_for_acc.append(filtered_df)

                        if not have:
                            continue
                            
                        df_concat = pd.concat(dataframes_for_acc)

                        plt.plot(df_concat['total_communication_gb'], df_concat['accuracy'], marker='o', color=base_colors[fda_name], label=fda_name, markersize=3)

                    synchronous_df = df_bias[(df_bias['num_clients'] == client) & (df_bias['fda_name'] == 'synchronous')]
                    
                    for b in sorted(synchronous_df['batch_size'].unique()):

                        sync_df = synchronous_df[synchronous_df['batch_size'] == b]

                        if sync_df.empty:
                            continue

                        dataframes_for_acc = []
                        
                        have = False

                        for acc in accuracies:

                            filtered_df = sync_df[(sync_df.accuracy > acc)]

                            if filtered_df.empty:
                                continue
                                
                            have = True

                            idx = filtered_df['epoch'].idxmin()
                            filtered_df = filtered_df.loc[idx]

                            dataframes_for_acc.append(filtered_df)

                            
                        if not have:
                            continue 
                            
                        df_concat = pd.concat(dataframes_for_acc)

                        plt.plot(df_concat['total_communication_gb'], df_concat['accuracy'], color=sync_colors[b], marker='o', label=f'synchronous b{b}', markersize=3)

                    plt.xscale('log')
                    plt.ylabel('Accuracy')
                    plt.legend()
                    plt.xlabel('Communication Cost (GB)')
                    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                    title = f'$Theta$ : {theta} , Batch Size: {bs} , {bias_title} , Num Clients: {client}'
                    plt.title(title)
                    plt.suptitle(fda_name)

                    pdf.savefig(plt.gcf())

                    plt.close()
    pdf.close()

In [55]:
def per_layer_invest_fda_methods_clients_comm_cost(df, filename):
    pdf = PdfPages(filename)
    
    batch_size_values = [32]
    theta_values = [100.]
    
    biases = [np.NaN]
    
    per_layers = [True, False]

    for bias in biases:
        
        if pd.isna(bias):
            mask = df['bias'].isna()
            bias_title = f'Bias: {bias}'
        elif bias > 0:
            mask = df['bias'] == bias
            bias_title = f'Bias: {bias}'
        else:
            mask = df['bias'] == bias
            bias_title = f'Bias: only label {one_label_bias[bias]}'
            
        df_bias = df[mask]
        
        for batch_size in batch_size_values:
            for theta in theta_values:

                filtered_df = df_bias[(df_bias['theta'] == theta) & (df_bias['batch_size'] == batch_size)] 

                if filtered_df.empty:
                    continue

                plt.figure(figsize=(10, 6))
                
                for per_layer in per_layers:
                    
                    per_layer_df = filtered_df[filtered_df['per_layer'] == per_layer]
                    # --------------------------------------------------------------------
                    for fda_name in fda_names:
                        
                        name = f"{fda_name}{per_layer_dict[per_layer]}"

                        if fda_name == 'synchronous':
                            continue

                        fda_data = per_layer_df[per_layer_df['fda_name'] == fda_name]

                        if fda_data.empty:
                            continue

                        plt.plot(fda_data['num_clients'], fda_data['total_communication_gb'], marker='o', color=base_colors_per_layer[name], label=name, markersize=3)

                synchronous_data = df_bias[(df_bias['batch_size'] == batch_size) & (df_bias['fda_name'] == 'synchronous')] 

                if not synchronous_data.empty:
                    plt.plot(synchronous_data['num_clients'], synchronous_data['total_communication_gb'], marker='o', label='synchronous', color=base_colors['synchronous'], markersize=3)

                # Set xticks every 5 units based on available values
                min_clients = 5
                max_clients = 60

                x_ticks = list(range(min_clients, max_clients + 1, 5))

                plt.yscale('log')
                plt.xticks(x_ticks)
                plt.xlabel('Number of clients')
                plt.legend()
                plt.ylabel('Communication (GB)')
                plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

                title = f'Batch Size : {batch_size} , $\Theta$ : {theta} , {bias_title}'
                plt.suptitle(title)

                plt.tight_layout()

                pdf.savefig(plt.gcf()) # Save the current figure
        
                # Close the current figure to prevent it from being displayed in the notebook
                plt.close()
    pdf.close()

In [138]:
def per_layer_tot_rounds(df, filename):
    pdf = PdfPages(filename)
    
    df_per_layer = df[df.per_layer]
    clients = list(range(5, 60 + 1, 5))
    epochs = [(0, 50), (50, 100)]
    
    s = sum(adv_layer_count)
    n = len(adv_layer_count)
    adv_perc_layer_w = [100 * c_w / s for c_w in adv_layer_count]
    
    for c in clients:
        df_c = df_per_layer[df_per_layer.num_clients == c]
            
        plt.figure(figsize=(10, 6))

        for layer_i, layer_perc in enumerate(adv_perc_layer_w):

            plt.plot(df_c['epoch'], df_c['total_rounds'].str[layer_i], marker='o', label=f'layer w/ {layer_perc:.1f}%', markersize=3)


        plt.xlabel('Epoch')
        plt.legend()
        plt.ylabel('Total Rounds')
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

        title = f'Batch Size : 32 , $\Theta$ : 100 , bias : NaN, Num Clients : {c}'
        plt.suptitle(title)

        pdf.savefig(plt.gcf()) # Save the current figure

        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
        
    pdf.close()

In [144]:
def per_layer_rounds(df, filename):
    pdf = PdfPages(filename)
    
    df_per_layer = df[df.per_layer]
    clients = list(range(5, 60 + 1, 5))
    epochs = [(0, 50), (50, 100)]
    
    s = sum(adv_layer_count)
    n = len(adv_layer_count)
    adv_perc_layer_w = [100 * c_w / s for c_w in adv_layer_count]
    
    for c in clients:
        df_c = df_per_layer[df_per_layer.num_clients == c]
        
        df_e = df_c.copy()
        
        df_e['epoch_rounds'] = df_e['total_rounds'].diff() 
        df_e['epoch_rounds'] = df_e['epoch_rounds'].fillna(df_e['total_rounds'])
            
        plt.figure(figsize=(10, 6))

        for layer_i, layer_perc in enumerate(adv_perc_layer_w):

            plt.plot(df_e['epoch'], df_e['epoch_rounds'].str[layer_i], marker='o', label=f'layer w/ {layer_perc:.1f}%', markersize=3)


        plt.xlabel('Epoch')
        plt.legend()
        plt.ylabel('Rounds in Epoch')
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)

        title = f'Batch Size : 32 , $\Theta$ : 100 , bias : NaN, Num Clients : {c}'
        plt.suptitle(title)

        pdf.savefig(plt.gcf()) # Save the current figure

        # Close the current figure to prevent it from being displayed in the notebook
        plt.close()
        
    pdf.close()

# Help-Stat

In [56]:
def explore_top(df, acc_thresh, nn_name, fda_name):
    acceptable_acc_df = df[(df.accuracy > acc_thresh) & (df.nn_name == nn_name)]
    fda_df = acceptable_acc_df[acceptable_acc_df['fda_name'] == fda_name]
    
    idx = fda_df.groupby(['fda_name', 'num_clients', 'bias', 'batch_size', 'theta', 'aggr_scheme', 'per_layer'], dropna=False)['epoch'].idxmin()
    aggr_df = fda_df.loc[idx]
    
    print(len(aggr_df))
    
    aggr_df['monitoring_gb_exchanged'] = aggr_df['monitoring_bytes_exchanged'] / 10**9
    aggr_df['model_gb_exchanged'] = aggr_df['model_bytes_exchanged'] / 10**9
    
    #return filtered_acceptable_acc_df
    return aggr_df[['num_clients', 'batch_size', 'theta', 'bias', 'per_layer', 'total_rounds', 'total_fda_steps', 'epoch', 'monitoring_gb_exchanged', 'model_gb_exchanged', 'total_communication_gb', 'cpu_time_cost', 'hypercube_time_cost', 'common_channel_time_cost']].sort_values(by='common_channel_time_cost')

## Save all those time-cost plots

In [88]:
import os

def time_cost_plots(df, acc_threshold, nn_name, limit_x_axis=False, show_runs=False, params=False, kde_time_log=True, kde_comm_log=True, kde_cpu_log=False):
    # Filter out based on `acc_threshold`
    df_nn = df[(df.nn_name == nn_name)]
    
    acceptable_acc_df = df_nn[(df_nn.accuracy > acc_threshold)]
    
    str_thresh = str(acc_threshold).replace('.', '_')  # replace '.'
    
    if not os.path.exists(f"../../metrics/plots/{nn_name}/{str_thresh}"):
        os.makedirs(f"../../metrics/plots/{nn_name}/{str_thresh}")
        
    # 1. Same runs not included
    
    # 2. Filter out same runs. We choose the instance which first hits the `acc_threshold`
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta', 'bias', 'aggr_scheme', 'per_layer'], dropna=False)['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    
    kde_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/time_cost.pdf", x_log=kde_time_log)
    
    kde_communication_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/comm_cost.pdf", x_log=kde_comm_log)
    
    kde_cpu_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/cpu_cost.pdf", x_log=kde_cpu_log)
    
    kde_comm_cpu_time_cost_ratio(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/ratio.pdf", x_log=kde_comm_log)
    
    # Parameters fixed, and plot
    if params:
        fda_methods_clients_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_time_cost.pdf")
        fda_methods_clients_comm_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_comm_cost.pdf")
        fda_methods_clients_cpu_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/clients_cpu_cost.pdf")
        
        fda_methods_theta_time_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/theta_time_cost.pdf")
        fda_methods_theta_comm_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/theta_comm_cost.pdf")
        fda_methods_theta_cpu_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/theta_cpu_cost.pdf")

In [148]:
def testing(df, acc_threshold, nn_name):
    df_nn = df[(df.nn_name == nn_name)]
    
    acceptable_acc_df = df_nn[(df_nn.accuracy > acc_threshold)]
    
    str_thresh = str(acc_threshold).replace('.', '_')  # replace '.'
    
    if not os.path.exists(f"../../metrics/plots/{nn_name}/{str_thresh}"):
        os.makedirs(f"../../metrics/plots/{nn_name}/{str_thresh}")
        
    idx = acceptable_acc_df.groupby(['fda_name', 'num_clients', 'batch_size', 'theta', 'bias', 'aggr_scheme', 'per_layer'], dropna=False)['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    
    # Temporary per-layer
    per_layer_invest_fda_methods_clients_comm_cost(filtered_acceptable_acc_df, f"../../metrics/plots/{nn_name}/{str_thresh}/per_layer_clients_comm_cost.pdf")
    
    per_layer_rounds(df, f"../../metrics/plots/{nn_name}/rounds_per_epoch.pdf")
    
    per_layer_tot_rounds(df, f"../../metrics/plots/{nn_name}/total_rounds_per_epoch.pdf")

In [75]:
df_32 = df[
    ((df['batch_size'] == 32) | (df['fda_name'] == 'synchronous')) & (df['aggr_scheme'] == 'avg')
]

In [76]:
df_theta = df[
    ((df['theta'] >= 15.) | (df['fda_name'] == 'synchronous')) & (df['aggr_scheme'] == 'avg')
]

In [86]:
df_adv = df_theta[~df_theta.per_layer]
df_lenet = df_32[~df_32.per_layer]

## AdvancedCNN

In [89]:
time_cost_plots(df_adv, 0.99, 'AdvancedCNN', params=True)
time_cost_plots(df_adv, 0.993, 'AdvancedCNN', params=True)
time_cost_plots(df_adv, 0.995, 'AdvancedCNN', params=True)

In [ ]:
accuracies_plots_clients(df_adv[df_adv['nn_name'] == 'AdvancedCNN'], f"../../metrics/plots/AdvancedCNN/accuracies_clients.pdf", accuracies=[0.98, 0.985, 0.99, 0.991, 0.992, 0.993, 0.994, 0.995, 0.996])

In [ ]:
accuracies_plots(df_adv[df_adv['nn_name'] == 'AdvancedCNN'], f"../../metrics/plots/AdvancedCNN/accuracies.pdf", accuracies=[0.98, 0.985, 0.99, 0.991, 0.992, 0.993, 0.994, 0.995, 0.996])

In [149]:
testing(df_theta, 0.99, 'AdvancedCNN')

## LeNet-5

In [ ]:
#explore_top(df, 0.98, 'LeNet-5', 'linear')

In [ ]:
time_cost_plots(df_lenet, 0.98, 'LeNet-5', params=True)
time_cost_plots(df_lenet, 0.975, 'LeNet-5', params=True)
time_cost_plots(df_lenet, 0.985, 'LeNet-5', params=True)

In [ ]:
accuracies_plots_clients(df_lenet[df_lenet['nn_name'] == 'LeNet-5'], f"../../metrics/plots/LeNet-5/accuracies_clients.pdf", accuracies=[0.96, 0.965, 0.96, 0.965, 0.98, 0.982, 0.984, 0.985, 0.987, 0.989])

In [ ]:
accuracies_plots(df_lenet[df_lenet['nn_name'] == 'LeNet-5'], f"../../metrics/plots/LeNet-5/accuracies.pdf", accuracies=[0.96, 0.965, 0.96, 0.965, 0.98, 0.982, 0.984, 0.985, 0.987, 0.989])

In [ ]:
def investigate_rounds(df_nn, acc_threshold):
    acceptable_acc_df = df_nn[(df_nn.accuracy > acc_threshold)]
    idx = acceptable_acc_df.groupby(['dataset_name', 'nn_name', 'fda_name', 'num_steps_until_rtc_check', 'batch_size', 'theta', 'num_clients', 'aggr_scheme', 'bias'], dropna=False)['epoch'].idxmin()
    filtered_acceptable_acc_df = acceptable_acc_df.loc[idx]
    
    num_clients = sorted(filtered_acceptable_acc_df['num_clients'].unique())
    
    for num_client in num_clients:
        df_clients = filtered_acceptable_acc_df[filtered_acceptable_acc_df['num_clients'] == num_client]
        
        print(f"Clients: {num_client}  Avg Rounds Needed: {df_clients['total_rounds'].mean()}")

In [ ]:
investigate_rounds(df[(df['nn_name'] == 'AdvancedCNN') & (df['bias'].isna()) & (df['batch_size'] == 32) & (df['theta'] >= 15) & (df['fda_name'] == 'naive')], 0.995)

In [ ]:
investigate_rounds(df[(df['nn_name'] == 'AdvancedCNN') & (df['bias'].isna()) & (df['batch_size'] == 32) & (df['theta'] >= 50) & (df['fda_name'] == 'naive')], 0.995)

In [ ]:
investigate_rounds(df[(df['nn_name'] == 'AdvancedCNN') & (df['bias'].isna()) & (df['batch_size'] == 32) & (df['theta'] >= 50) & (df['fda_name'] == 'sketch') & (df['aggr_scheme'] == 'avg')], 0.995)